In [43]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# from io import StringIO # python3; python2: BytesIO 
import boto3
import s3fs
import awscli
import timeit
from datetime import datetime

In [44]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

team3-final-bucket
wshih-bucket


In [45]:
my_bucket = s3.Bucket('team3-final-bucket')

In [62]:
client = boto3.client('s3')
path = 's3://team3-final-bucket/Resources_clean/'

In [63]:
objects_df = pd.read_csv(path + 'objects_clean.csv', dtype={'status':str, 'entity_id':str})
objects_df.head()

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


In [64]:
objects_df.dtypes

Unnamed: 0               int64
object_id               object
entity_type             object
parent_id               object
name                    object
category_code           object
status                  object
founded_at              object
closed_at               object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_total_usd      float64
first_milestone_at      object
last_milestone_at       object
milestones               int64
relationships            int64
dtype: object

In [65]:
objects_df['category_code'].unique()

array(['web', 'games_video', 'network_hosting', 'advertising',
       'cleantech', nan, 'enterprise', 'other', 'consulting', 'mobile',
       'health', 'software', 'analytics', 'finance', 'education',
       'medical', 'manufacturing', 'biotech', 'ecommerce',
       'public_relations', 'hardware', 'search', 'news', 'government',
       'security', 'photo_video', 'travel', 'semiconductor', 'social',
       'legal', 'transportation', 'hospitality', 'sports', 'nonprofit',
       'fashion', 'messaging', 'music', 'automotive', 'design',
       'real_estate', 'local', 'nanotech', 'pets'], dtype=object)

In [66]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [67]:
is_string_dtype(objects_df['status'])

True

In [68]:
is_numeric_dtype(objects_df['status'])

False

In [69]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(objects_df.dtypes)

Unnamed: 0               int64
object_id               object
entity_type             object
parent_id               object
name                    object
category_code           object
status                  object
founded_at              object
closed_at               object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_total_usd      float64
first_milestone_at      object
last_milestone_at       object
milestones               int64
relationships            int64
dtype: object


In [70]:
objects_df.head()

,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


##### Transforming Data

In [71]:
objects_df = objects_df.rename(columns = {'id':'object_id'})
objects_df.head()

,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


##### Converting date columns into date format

In [72]:
# Converting strings to date type
objects_clean['founded_at'] = pd.to_datetime(objects_clean.founded_at, format='%Y-%m-%d')
objects_clean['closed_at'] = pd.to_datetime(objects_clean.closed_at, format='%Y-%m-%d')
objects_clean['first_investment_at'] = pd.to_datetime(objects_clean.first_investment_at, format='%Y-%m-%d')
objects_clean['last_investment_at'] = pd.to_datetime(objects_clean.last_investment_at, format='%Y-%m-%d')
objects_clean['first_funding_at'] = pd.to_datetime(objects_clean.first_funding_at, format='%Y-%m-%d')
objects_clean['last_funding_at'] = pd.to_datetime(objects_clean.last_funding_at, format='%Y-%m-%d')
objects_clean['first_milestone_at'] = pd.to_datetime(objects_clean.first_milestone_at, format='%Y-%m-%d')
objects_clean['last_milestone_at'] = pd.to_datetime(objects_clean.last_milestone_at, format='%Y-%m-%d')

In [73]:
objects_clean.dtypes

object_id                      object
entity_type                    object
entity_id                      object
parent_id                      object
name                           object
category_code                  object
status                         object
founded_at             datetime64[ns]
closed_at              datetime64[ns]
country_code                   object
state_code                     object
city                           object
region                         object
first_investment_at    datetime64[ns]
last_investment_at     datetime64[ns]
investment_rounds               int64
invested_companies              int64
first_funding_at       datetime64[ns]
last_funding_at        datetime64[ns]
funding_rounds                  int64
funding_total_usd             float64
first_milestone_at     datetime64[ns]
last_milestone_at      datetime64[ns]
milestones                      int64
relationships                   int64
end_dates                      object
dtype: objec

In [74]:
# Changing columns order
objects_clean.columns.to_list()

['object_id',
 'entity_type',
 'entity_id',
 'parent_id',
 'name',
 'category_code',
 'status',
 'founded_at',
 'closed_at',
 'country_code',
 'state_code',
 'city',
 'region',
 'first_investment_at',
 'last_investment_at',
 'investment_rounds',
 'invested_companies',
 'first_funding_at',
 'last_funding_at',
 'funding_rounds',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships',
 'end_dates']

In [96]:
# Adding new column for collecting start dates

objects_clean["start_dates"] = ""

In [97]:
# Adding a new column for collecting end dates

objects_clean["end_dates"] = ""
objects_clean.head()

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,end_dates,start_dates
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaT,USA,...,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,,
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,USA,...,NaT,NaT,0,0.0,NaT,NaT,0,6,,
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,USA,...,NaT,NaT,0,0.0,2003-02-01,2011-09-23,4,12,,
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaN,...,NaT,NaT,0,0.0,NaT,NaT,0,0,,
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaN,...,NaT,NaT,0,0.0,NaT,NaT,0,0,,


In [99]:
objects_clean = objects_clean[['object_id',
 'entity_type',
 'entity_id',
 'parent_id',
 'name',
 'category_code',
 'status',
 'start_dates',
 'founded_at',
 'first_investment_at',
 'first_funding_at',
 'country_code',
 'state_code',
 'city',
 'region',
 'investment_rounds',
 'invested_companies',
 'funding_rounds',
 'end_dates',                               
 'closed_at',
 'last_investment_at',
 'last_funding_at',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships']]

In [100]:
objects_clean.head()

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,founded_at,first_investment_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,,2005-10-17,NaT,...,3,,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,,2008-07-26,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,,2008-07-26,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [ ]:
#objects_clean2.dtypes

In [101]:
# Filtering 'companies' only
companies_df = objects_clean[objects_clean['entity_type']=='Company']
companies_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,founded_at,first_investment_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,,2005-10-17,NaT,...,3,,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,,2008-07-26,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,,2008-07-26,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,,2007-01-01,NaT,...,0,,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,,2007-11-01,NaT,...,1,,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,,1959-01-01,NaT,...,0,,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,,2008-07-01,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,1


##### Working on filling in gap of start and end dates

In [102]:
# Filling out start dates gaps by combining dates from three columns of 'Companies':
# funded_at, first_investment_at, first_funding_at.
# If conflicting dates are found in one record, the earliest is taken.

def mindate(a,b,c):
    return min(a,b,c)

companies_df['start_dates'] = companies_df.apply(lambda row : mindate(row['founded_at'], 
                     row['first_investment_at'], row['first_funding_at']), axis = 1)

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [103]:
def maxdate(a,b,c):
    if a == 'NaT':
        return max(b,c)
    elif b == 'NaT':
        return max(a,c)
    else:
        return c


In [104]:
# function returns 'NaT' as max value. Therefore have to give arbitrary max date to compare to

companies_df['end_dates'] = companies_df.apply(lambda row : maxdate(row['closed_at'], 
                      row['last_investment_at'], row['last_funding_at']), axis = 1)

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [105]:
companies_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,founded_at,first_investment_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-01,2005-10-17,NaT,...,3,2008-05-19,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,2008-07-26,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,2008-07-26,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,2007-01-01,2007-01-01,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,2007-11-01,NaT,...,1,2008-08-25,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,1959-01-01,1959-01-01,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,2008-07-01,2008-07-01,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,1


In [106]:
# Dropping columns
cleaned_companies_df = companies_df.drop(companies_df[['founded_at','first_investment_at','first_funding_at','closed_at','last_investment_at','last_funding_at']], axis=1)
cleaned_companies_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,country_code,state_code,...,region,investment_rounds,invested_companies,funding_rounds,end_dates,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-01,USA,WA,...,Seattle,0,0,3,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,USA,CA,...,Los Angeles,0,0,0,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,USA,CA,...,SF Bay,0,0,0,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,unknown,0,0,0,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,unknown,0,0,0,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,2007-01-01,USA,CA,...,SF Bay,0,0,0,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,USA,CA,...,SF Bay,0,0,1,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,1959-01-01,USA,VA,...,Washington DC,0,0,0,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,2008-07-01,USA,CA,...,Los Angeles,0,0,0,NaT,0.0,NaT,NaT,0,1


In [107]:
# Counting notnulls in end_dates
cleaned_companies_df[cleaned_companies_df['start_dates'].notna()]

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,country_code,state_code,...,region,investment_rounds,invested_companies,funding_rounds,end_dates,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-01,USA,WA,...,Seattle,0,0,3,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,unknown,0,0,0,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,unknown,0,0,0,NaT,0.0,NaT,NaT,0,0
5,c:10002,Company,10002,NaN,Synergie Media,advertising,operating,2007-06-27,MAR,NaN,...,Agadir,0,0,0,NaT,0.0,NaT,NaT,0,2
6,c:10003,Company,10003,NaN,Green Basti Ecosystems,cleantech,operating,2008-08-20,IND,NaN,...,Vadodara,0,0,0,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,2007-01-01,USA,CA,...,SF Bay,0,0,0,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,USA,CA,...,SF Bay,0,0,1,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,1959-01-01,USA,VA,...,Washington DC,0,0,0,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,2008-07-01,USA,CA,...,Los Angeles,0,0,0,NaT,0.0,NaT,NaT,0,1


In [108]:
# Counting notnulls in end_dates
cleaned_companies_df[cleaned_companies_df['end_dates'].notna()]

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,country_code,state_code,...,region,investment_rounds,invested_companies,funding_rounds,end_dates,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-01,USA,WA,...,Seattle,0,0,3,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
13,c:1001,Company,1001,NaN,FriendFeed,web,acquired,2007-10-01,USA,CA,...,SF Bay,0,0,1,2008-02-26,5000000.0,2008-05-01,2012-09-13,3,14
18,c:10014,Company,10014,NaN,Mobclix,mobile,acquired,2008-03-01,USA,CA,...,SF Bay,0,0,1,2008-09-01,0.0,1995-03-01,2012-09-18,4,9
19,c:10015,Company,10015,NaN,Fitbit,health,operating,2007-10-01,USA,CA,...,SF Bay,0,0,5,2013-08-13,68069200.0,NaT,NaT,0,14
20,c:100155,Company,100155,NaN,MTPV,cleantech,operating,2003-01-01,USA,TX,...,Austin,0,0,3,2012-01-26,10125293.0,2010-01-01,2010-01-01,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196537,c:99853,Company,99853,NaN,ChatterBlock,social,operating,2011-01-11,CAN,NaN,...,Victoria,0,0,1,2011-09-06,500000.0,2011-01-11,2011-01-11,1,2
196540,c:9989,Company,9989,NaN,Shattered Reality Interactive,games_video,closed,2006-06-01,USA,CA,...,SF Bay,0,0,2,2009-02-01,1100000.0,2005-11-01,2008-09-10,2,4
196547,c:9994,Company,9994,NaN,Imindi,advertising,operating,2007-01-01,NaN,NaN,...,unknown,0,0,2,2008-05-01,500000.0,2009-08-27,2009-08-27,1,2
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,USA,CA,...,SF Bay,0,0,1,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14


##### There are only 31507 end date records from the companies_df
-----

#### Looking at Product from original df for start and end dates

In [109]:
# Looking at products from the original df for end dates

end_products_df = objects_clean[objects_clean['entity_type']=='Product']
end_products_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,founded_at,first_investment_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
434913,r:1,Product,1,c:1,Wikison Wetpaint,NaN,live,,2006-06-01,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434914,r:10,Product,10,c:16,Slacker Web Player,NaN,operating,,2007-06-05,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434915,r:100,Product,100,c:1585,Vemo Enterprise Workforce Planning,NaN,live,,2008-07-01,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434916,r:10002,Product,10002,c:14171,dexmo.com - study and revision network,NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434917,r:10003,Product,10003,c:14173,VIrtual PBX Parachute,NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,9995,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462647,r:9996,Product,9996,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462648,r:9997,Product,9997,c:14164,"Cmail, broadcast email marketing",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462649,r:9998,Product,9998,c:14164,"CatylistCRM, contact database",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [110]:
# Dropping null values in parent_id
cleaned_end_products_df = end_products_df[end_products_df['parent_id'].notnull()]
cleaned_end_products_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,start_dates,founded_at,first_investment_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
434913,r:1,Product,1,c:1,Wikison Wetpaint,NaN,live,,2006-06-01,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434914,r:10,Product,10,c:16,Slacker Web Player,NaN,operating,,2007-06-05,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434915,r:100,Product,100,c:1585,Vemo Enterprise Workforce Planning,NaN,live,,2008-07-01,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434916,r:10002,Product,10002,c:14171,dexmo.com - study and revision network,NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434917,r:10003,Product,10003,c:14173,VIrtual PBX Parachute,NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,9995,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462647,r:9996,Product,9996,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462648,r:9997,Product,9997,c:14164,"Cmail, broadcast email marketing",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462649,r:9998,Product,9998,c:14164,"CatylistCRM, contact database",NaN,operating,,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [111]:
cleaned_end_products_df['closed_at'].unique()

array([                          'NaT', '2007-09-21T00:00:00.000000000',
       '2010-01-01T00:00:00.000000000', '2008-11-14T00:00:00.000000000',
       '2009-04-01T00:00:00.000000000', '2011-09-20T00:00:00.000000000',
       '1968-01-22T00:00:00.000000000', '2012-03-01T00:00:00.000000000',
       '2009-04-27T00:00:00.000000000', '2009-01-01T00:00:00.000000000',
       '2011-05-01T00:00:00.000000000', '2009-10-21T00:00:00.000000000',
       '2012-10-01T00:00:00.000000000', '2009-01-07T00:00:00.000000000',
       '2008-05-02T00:00:00.000000000', '2012-01-01T00:00:00.000000000',
       '2013-12-30T00:00:00.000000000', '2011-01-01T00:00:00.000000000',
       '1999-11-06T00:00:00.000000000', '2007-01-12T00:00:00.000000000',
       '2009-09-21T00:00:00.000000000', '2009-03-25T00:00:00.000000000',
       '2011-11-16T00:00:00.000000000', '2008-10-26T00:00:00.000000000',
       '2012-01-05T00:00:00.000000000', '2008-06-27T00:00:00.000000000',
       '2011-03-31T00:00:00.000000000', '1968-07-05

In [112]:
cleaned_end_products_df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['object_id', 'entity_type', 'entity_id', 'parent_id', 'name',
       'category_code', 'status', 'start_dates', 'founded_at',
       'first_investment_at', 'first_funding_at', 'country_code', 'state_code',
       'city', 'region', 'investment_rounds', 'invested_companies',
       'funding_rounds', 'end_dates', 'closed_at', 'last_investment_at',
       'last_funding_at', 'funding_total_usd', 'first_milestone_at',
       'last_milestone_at', 'milestones', 'relationships'],
      dtype='object')>

In [88]:
# Dropping all redundant columns

ce2_products_df = cleaned_end_products_df.drop(['entity_type', 'entity_id', 
       'category_code', 'founded_at', 'first_investment_at', 'first_funding_at', 'country_code', 'state_code',
       'city', 'region', 'investment_rounds', 'invested_companies',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at', 'last_milestone_at',
       'milestones', 'relationships'], axis=1)

ce2_products_df

,object_id,parent_id,name,status,end_dates,closed_at,last_investment_at,last_funding_at
434913,r:1,c:1,Wikison Wetpaint,live,,NaT,NaT,NaT
434914,r:10,c:16,Slacker Web Player,operating,,NaT,NaT,NaT
434915,r:100,c:1585,Vemo Enterprise Workforce Planning,live,,NaT,NaT,NaT
434916,r:10002,c:14171,dexmo.com - study and revision network,operating,,NaT,NaT,NaT
434917,r:10003,c:14173,VIrtual PBX Parachute,operating,,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...
462646,r:9995,c:14164,"SiteLink, listing feed for Brokerages",operating,,NaT,NaT,NaT
462647,r:9996,c:14164,"EDCLink, listing feed for Economic Development...",operating,,NaT,NaT,NaT
462648,r:9997,c:14164,"Cmail, broadcast email marketing",operating,,NaT,NaT,NaT
462649,r:9998,c:14164,"CatylistCRM, contact database",operating,,NaT,NaT,NaT


In [89]:
ce2_products_df['last_investment_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [90]:
ce2_products_df['last_funding_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [91]:
# Dropping last_investment_at and last_funding_at columns

products_df_final = ce2_products_df.drop(['last_investment_at', 'last_funding_at'], axis=1)
products_df_final

,object_id,parent_id,name,status,end_dates,closed_at
434913,r:1,c:1,Wikison Wetpaint,live,,NaT
434914,r:10,c:16,Slacker Web Player,operating,,NaT
434915,r:100,c:1585,Vemo Enterprise Workforce Planning,live,,NaT
434916,r:10002,c:14171,dexmo.com - study and revision network,operating,,NaT
434917,r:10003,c:14173,VIrtual PBX Parachute,operating,,NaT
...,...,...,...,...,...,...
462646,r:9995,c:14164,"SiteLink, listing feed for Brokerages",operating,,NaT
462647,r:9996,c:14164,"EDCLink, listing feed for Economic Development...",operating,,NaT
462648,r:9997,c:14164,"Cmail, broadcast email marketing",operating,,NaT
462649,r:9998,c:14164,"CatylistCRM, contact database",operating,,NaT


In [92]:
products_df_final = products_df_final[products_df_final['closed_at'].notna()]
products_df_final

,object_id,parent_id,name,status,end_dates,closed_at
435039,r:10173,c:14409,Friendzbook,closed,,2007-09-21
435064,r:102,c:460,Sticky Notes,closed,,2010-01-01
435118,r:10253,c:3347,ZuiGames,closed,,2008-11-14
435229,r:10393,c:14624,Bitloot,closed,,2009-04-01
435236,r:104,c:112,Zlio,closed,,2011-09-20
...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,,1993-05-12
461911,r:9055,c:12152,MacFeeder Sverige,closed,,2009-06-01
462096,r:9291,c:12,Twicco,closed,,2009-07-31
462124,r:9320,c:13193,WW.DataServices,closed,,2007-03-01


##### Only 189 more "end_dates" would be available from the products_df

In [94]:
# Checking funding_rounds_clean.csv for more end dates

fr_df = pd.read_csv(path + 'funding_rounds_clean.csv')
fr_df.head()

,Unnamed: 0,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round
0,0,1,c:4,12/1/2006,series-b,b,8500000,8500000,USD,0,0,NaN,0,0,NaN,2,0,0
1,1,2,c:5,9/1/2004,angel,angel,500000,500000,USD,0,0,USD,0,0,USD,2,0,1
2,2,3,c:5,5/1/2005,series-a,a,12700000,12700000,USD,115000000,115000000,USD,0,0,USD,3,0,0
3,3,4,c:5,4/1/2006,series-b,b,27500000,27500000,USD,525000000,525000000,USD,0,0,USD,4,0,0
4,4,5,c:7299,5/1/2006,series-b,b,10500000,10500000,USD,0,0,NaN,0,0,NaN,2,0,0


In [95]:
fr_df.columns.to_list()

['Unnamed: 0',
 'funding_round_id',
 'object_id',
 'funded_at',
 'funding_round_type',
 'funding_round_code',
 'raised_amount_usd',
 'raised_amount',
 'raised_currency_code',
 'pre_money_valuation_usd',
 'pre_money_valuation',
 'pre_money_currency_code',
 'post_money_valuation_usd',
 'post_money_valuation',
 'post_money_currency_code',
 'participants',
 'is_first_round',
 'is_last_round']

##### There doesn't seem to be any column that would provide us with end dates 
I would need to compare the funded_at column with the start dates from Irina's notebook to see if there is any overlap. I'm thinking that the funded_at cloumns will have a later date than the start dates.